In [ ]:
pip install tensor

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [6]:
def load_coffee_data():
    """ Creates a coffee roasting data set.
        roasting duration: 12-15 minutes is best
        temperature range: 175-260C is best
    """
    rng = np.random.default_rng(2)
    X = rng.random(400).reshape(-1,2)
    X[:,1] = X[:,1] * 4 + 11.5          # 12-15 min is best
    X[:,0] = X[:,0] * (285-150) + 150  # 350-500 F (175-260 C) is best
    Y = np.zeros(len(X))

    i=0
    for t,d in X:
        y = -3/(260-175)*t + 21
        if (t > 175 and t < 260 and d > 12 and d < 15 and d<=y ):
            Y[i] = 1
        else:
            Y[i] = 0
        i += 1

    return (X, Y.reshape(-1,1))

In [8]:
X_train,y_train=load_coffee_data()
print(X_train.shape,y_train.shape)

(200, 2) (200, 1)


*Applying Normalization*

Step1: Create a 'Normalization Layer'(This isnt a layer in our model)

Step2: 'adapt' the data. This learns the mean and variance of the data set and saves the values internally.

Step3: Normalize the data.


**It is important to apply normalization to any future data that utilizes the learned model.**

In [ ]:
print(f'Data Before Normalization \n{X_train}')

In [19]:
#In the context of tf.keras.layers.Normalization, setting axis=-1 means that the
#normalization operation will be applied along the last axis of the input tensor
norm_layer=tf.keras.layers.Normalization(axis=-1);
norm_layer.adapt(X_train) #Learns Mean and Variance

In [20]:
X_norm=norm_layer(X_train)

In [ ]:
print(f'Data After Normalization \n{X_norm}')

In [24]:
#Tile/copy our data to increase the training set size and reduce the number of training epochs.
Xt = np.tile(X_norm,(1000,1))
Yt= np.tile(y_train,(1000,1))
print(Xt.shape, Yt.shape)


(200000, 2) (200000, 1)


## **Model Building**

In [25]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model=Sequential([
    tf.keras.Input(shape=(2,)),
    Dense(3,activation='sigmoid',name='layer1'),
    Dense(1,activation='sigmoid',name='layer2')
    ])

Note 1: The tf.keras.Input(shape=(2,)), specifies the expected shape of the input. This allows Tensorflow to size the weights and bias parameters at this point. This is useful when exploring Tensorflow models. This statement can be omitted in practice and Tensorflow will size the network parameters when the input data is specified in the model.fit statement.

Note 2: Including the sigmoid activation in the final layer is not considered best practice. It would instead be accounted for in the loss which improves numerical stability.


In [27]:
#Providing a description of network
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (None, 3)                 9         
                                                                 
 layer2 (Dense)              (None, 1)                 4         
                                                                 
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________



1.   The model.compile statement defines a loss function and specifies a compile optimization.

2.   The model.fit statement runs gradient descent and fits the weights to the data.



In [28]:
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
)

model.fit(
    Xt,Yt,
    epochs=10,
)

Epoch 1/10
6250/6250 [==============================] - 7s 1ms/step - loss: 0.1780
Epoch 2/10
6250/6250 [==============================] - 7s 1ms/step - loss: 0.1220
Epoch 3/10
6250/6250 [==============================] - 7s 1ms/step - loss: 0.1011
Epoch 4/10
6250/6250 [==============================] - 8s 1ms/step - loss: 0.0237
Epoch 5/10
6250/6250 [==============================] - 9s 1ms/step - loss: 0.0132
Epoch 6/10
6250/6250 [==============================] - 7s 1ms/step - loss: 0.0089
Epoch 7/10
6250/6250 [==============================] - 8s 1ms/step - loss: 0.0063
Epoch 8/10
6250/6250 [==============================] - 8s 1ms/step - loss: 0.0045
Epoch 9/10
6250/6250 [==============================] - 7s 1ms/step - loss: 0.0032
Epoch 10/10
6250/6250 [==============================] - 7s 1ms/step - loss: 0.0024


# **Predictions**

In [30]:
X_test = np.array([
    [200,13.9],  # postive example
    [200,17]])   # negative example
X_testn = norm_layer(X_test)
predictions = model.predict(X_testn)
print("predictions = \n", predictions)

1/1 [==============================] - 0s 57ms/step
predictions = 
 [[9.8697132e-01]
 [4.7937636e-08]]


In [31]:
yhat = np.zeros_like(predictions)
for i in range(len(predictions)):
    if predictions[i] >= 0.5:
        yhat[i] = 1
    else:
        yhat[i] = 0
print(f"decisions = \n{yhat}")

decisions = 
[[1.]
 [0.]]


Hence For [200,13.9] the coffee bean is still edible(since output is 1)

while for [200,17] its not